In [49]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [31]:
df= pd.read_csv("C:/Users/ADMIN/Downloads/APPLE_iPhone_SE.csv", usecols=['Reviews','Ratings'])

In [32]:
df.head()

,Ratings,Reviews
0,5,Great camera for pics and videos Battery life ...
1,5,Great device. Let me tell the Pros..1. Superb ...
2,5,"Who all loves older size i.e., 4.7 inch type s..."
3,5,This iPhone SE is the best phone ever you get....
4,5,This is my second iphone after iphone 4s. I’ve...


In [33]:
df['Ratings'].value_counts()

Ratings
5    6788
4    1717
3     535
1     474
2     199
Name: count, dtype: int64

In [76]:
def remove_pattern(input_text,pattern):
    r= re.findall(pattern, input_text)
    for i in r:
        input_text = re.sub(i, '', input_text)
    return input_text
                      

In [77]:
df['Cleaned_data'] = np.vectorize(remove_pattern)(df['Reviews'],"@[\w]*") 
df.head()


,Ratings,Reviews,Cleaned_data
0,5,Great camera for pics and videos Battery life ...,Great camera for pics and videos Battery life ...
1,5,Great device. Let me tell the Pros..1. Superb ...,Great device. Let me tell the Pros..1. Superb ...
2,5,"Who all loves older size i.e., 4.7 inch type s...","Who all loves older size i.e., 4.7 inch type s..."
3,5,This iPhone SE is the best phone ever you get....,This iPhone SE is the best phone ever you get....
4,5,This is my second iphone after iphone 4s. I’ve...,This is my second iphone after iphone 4s. I’ve...


In [78]:
df['Cleaned_data'] = df['Cleaned_data'].str.replace("[^a-zA-Z#]"," ")
df.head(10)

,Ratings,Reviews,Cleaned_data
0,5,Great camera for pics and videos Battery life ...,Great camera for pics and videos Battery life ...
1,5,Great device. Let me tell the Pros..1. Superb ...,Great device. Let me tell the Pros..1. Superb ...
2,5,"Who all loves older size i.e., 4.7 inch type s...","Who all loves older size i.e., 4.7 inch type s..."
3,5,This iPhone SE is the best phone ever you get....,This iPhone SE is the best phone ever you get....
4,5,This is my second iphone after iphone 4s. I’ve...,This is my second iphone after iphone 4s. I’ve...
5,4,Totally worth the price if:- you are a iOS fan...,Totally worth the price if:- you are a iOS fan...
6,4,Mobile is very handy. Packed with good perform...,Mobile is very handy. Packed with good perform...
7,4,As always compact and handy ... with good imag...,As always compact and handy ... with good imag...
8,4,If you are going to use apple for first time t...,If you are going to use apple for first time t...
9,3,I have Purchased The new Iphone SE 2020 on 20t...,I have Purchased The new Iphone SE 2020 on 20t...


In [127]:
df['Cleaned_data']=df['Cleaned_data'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df.head()

,Ratings,Reviews,Cleaned_data
0,5,Great camera for pics and videos Battery life ...,great camera pic video batteri life good with ...
1,5,Great device. Let me tell the Pros..1. Superb ...,great device. tell pros..1. superb build quali...
2,5,"Who all loves older size i.e., 4.7 inch type s...","love older size i.e., inch type should definit..."
3,5,This iPhone SE is the best phone ever you get....,thi iphon best phone ever get. beauti design g...
4,5,This is my second iphone after iphone 4s. I’ve...,thi second iphon after iphon i’v been long tim...


In [128]:
tokens=df['Cleaned_data'].apply(lambda x:x.split())
tokens.head()

0    [great, camera, pic, video, batteri, life, goo...
1    [great, device., tell, pros..1., superb, build...
2    [love, older, size, i.e.,, inch, type, should,...
3    [thi, iphon, best, phone, ever, get., beauti, ...
4    [thi, second, iphon, after, iphon, i’v, been, ...
Name: Cleaned_data, dtype: object

In [129]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokens = tokens.apply(lambda x: [stemmer.stem(i) for i in x])

In [130]:
for i in range(len(tokens)):
    tokens[i] = ' '.join(tokens[i]) 
df['Cleaned_data'] = tokens

In [131]:
#tfidf

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,accuracy_score

In [133]:
tfidf= TfidfVectorizer(max_features=20000,ngram_range=(1,5),analyzer='char')

In [134]:
X=tfidf.fit_transform(df['Cleaned_data'])
y=df['Ratings']

In [135]:
X.shape, y.shape

((9713, 20000), (9713,))

In [136]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0)

In [137]:
clf=LinearSVC(C=20, class_weight='balanced')
clf.fit(X_train,y_train)

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=20, class_weight='balanced')

In [139]:
y_pred=clf.predict(X_test)


In [140]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.46      0.40      0.43        93
           2       0.05      0.02      0.03        44
           3       0.23      0.21      0.22       119
           4       0.29      0.30      0.30       317
           5       0.80      0.82      0.81      1370

    accuracy                           0.66      1943
   macro avg       0.37      0.35      0.36      1943
weighted avg       0.65      0.66      0.66      1943



In [141]:
print(accuracy_score(y_test,y_pred))


0.6623777663407102


In [126]:
y_pred.count()

0    1943
dtype: int64

In [ ]:
y_pred.count()